<a href="https://colab.research.google.com/github/shiddema/Benz_Time_on_Bench/blob/main/benz_workbook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mercedes-Benz Greener Manufacturing
 
This was an assignment from a class. It makes good use of dimensionality reduction, scaling, encoding, pipelines, and ensemble model predictions. Specifically, here I used XGBoost's Regression model. 
 
I will discuss the problem and my thought processes along the way. I found the business case for this problem interesting. I am a fan of manufacturing and more specifically the theory of constraints. The description of the problem can be found below. 
 
 
---
 
 
 
 
 
 
**Reduce the time a Mercedes-Benz spends on the test bench.**
 
Problem Statement Scenario:
Since the first automobile, the Benz Patent Motor Car in 1886, Mercedes-Benz has stood for important automotive innovations. These include the passenger safety cell with a crumple zone, the airbag, and intelligent assistance systems. Mercedes-Benz applies for nearly 2000 patents per year, making the brand the European leader among premium carmakers. Mercedes-Benz is the leader in the premium car industry. With a huge selection of features and options, customers can choose the customized Mercedes-Benz of their dreams.
 
To ensure the safety and reliability of every unique car configuration before they hit the road, the company’s engineers have developed a robust testing system. As one of the world’s biggest manufacturers of premium cars, safety and efficiency are paramount on Mercedes-Benz’s production lines. However, optimizing the speed of their testing system for many possible feature combinations is complex and time-consuming without a powerful algorithmic approach.
 
You are required to reduce the time that cars spend on the test bench. Others will work with a dataset representing different permutations of features in a Mercedes-Benz car to predict the time it takes to pass testing. Optimal algorithms will contribute to faster testing, resulting in lower carbon dioxide emissions without reducing Mercedes-Benz’s standards.
 

In [ ]:
import pandas as pd

path = '/content/drive/MyDrive/Colab Notebooks/Data Files/benz_train.csv'

df = pd.read_csv(path)
df.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,...,X345,X346,X347,X348,X349,X350,X351,X352,X353,X354,X355,X356,X357,X358,X359,X360,X361,X362,X363,X364,X365,X366,X367,X368,X369,X370,X371,X372,X373,X374,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,...,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4209 entries, 0 to 4208
Columns: 378 entries, ID to X385
dtypes: float64(1), int64(369), object(8)
memory usage: 12.1+ MB


### Data Analysis
 
Already we can see a number of things. 
 
 
1.   Target Variable is nicely labeled "y". It is safe to assume that it is some unit of time. What we know for sure is that it is a continuous variable and therefore we know that we will be using some sort of regression to solve the problem. 
2.   There are both numerical and categorical features. These will have to be dealt with separately. The categorical features will require encoding, and the numerical ones, dimensionality reduction. 
3.   There is no description of data. Giving us very few options for understanding or manipulating the data. We will have to rely on variability. 
 

In [ ]:
X = df.iloc[:, df.columns != 'y']
Y = df.iloc[:, df.columns == 'y']
print(X.shape)
print(Y.shape)

(4209, 377)
(4209, 1)


In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4209 entries, 0 to 4208
Columns: 377 entries, ID to X385
dtypes: int64(369), object(8)
memory usage: 12.1+ MB


In [ ]:
X_cat = X.select_dtypes(include = 'object')
X_num = X.select_dtypes(include = 'int64')
print(X_cat.shape)
print(X_num.shape)

(4209, 8)
(4209, 369)


In [ ]:
X_num.var().sort_values()

X268    0.000000e+00
X297    0.000000e+00
X290    0.000000e+00
X289    0.000000e+00
X330    0.000000e+00
            ...     
X191    2.492121e-01
X362    2.496467e-01
X337    2.497867e-01
X127    2.500357e-01
ID      5.941936e+06
Length: 369, dtype: float64

So I have columns with zero variance. I want to remove those before moving on. I am going to set the threshold to zero. It is likely that I will want to remove more of the 377 columns later on. But we will leave that for another step. 

In [ ]:
from sklearn.feature_selection import VarianceThreshold
VT = VarianceThreshold()
X_num = VT.fit_transform(X_num)
X_num = pd.DataFrame(X_num)
print(X_num.shape)

(4209, 357)


In [ ]:
X_num.var().sort_values()

237    2.375861e-04
189    2.375861e-04
259    2.375861e-04
267    2.375861e-04
22     2.375861e-04
           ...     
174    2.492121e-01
334    2.496467e-01
310    2.497867e-01
112    2.500357e-01
0      5.941936e+06
Length: 357, dtype: float64

In [ ]:
X_cat.describe()

,X0,X1,X2,X3,X4,X5,X6,X8
count,4209,4209,4209,4209,4209,4209,4209,4209
unique,47,27,44,7,4,29,12,25
top,z,aa,as,c,d,v,g,j
freq,360,833,1659,1942,4205,231,1042,277


### Encoding the categorical features

I took this opportunity to explore some of the different encoders. This section is mostly trial and error to see which method would produce the best results for predicting our target variable. *Feel free to skip to the next section*. 

The steps will be to run the encoder, and then run XGBoost because XGBoost is what I will be using later on. If I was to do this again I would use a pipeline and nest the parameters, however, I wanted to do them manually this time. 

In [ ]:
# Rare Label Encoding -- setting n_categories to 3 so that all features will be addressed. 
from feature_engine.encoding import RareLabelEncoder
encoder = RareLabelEncoder(n_categories=3, replace_with='Rare')

# fit the encoder
X_cat = encoder.fit_transform(X_cat)
X_cat.describe()

,X0,X1,X2,X3,X4,X5,X6,X8
count,4209,4209,4209,4209,4209,4209,4209,4209
unique,9,7,6,6,2,8,6,8
top,Rare,Rare,as,c,d,Rare,g,Rare
freq,1761,937,1659,1942,4205,2672,1042,2511


In [ ]:
from feature_engine.encoding import OneHotEncoder, CountFrequencyEncoder, OrdinalEncoder, MeanEncoder, WoEEncoder, PRatioEncoder, DecisionTreeEncoder
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_cat, Y, random_state=42)

In [ ]:
# First up One Hot Encoder. This has two main params that we can play with: top_categories and drop_last

# set up the encoder
encoder = OneHotEncoder(drop_last=True)

# fit the encoder
encoder.fit(X_train)

# transform the data
train_t= encoder.transform(X_train)
test_t= encoder.transform(X_test)

In [ ]:
display(X_train.shape)
train_t.shape

(3156, 8)

(3156, 44)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)
pca.fit(train_t)
train_t = pca.transform(train_t)
pca.explained_variance_ratio_

array([0.09803531, 0.07803757, 0.06549249, 0.06042049, 0.05887551,
       0.05473915, 0.04535245, 0.04065842, 0.03945332, 0.03695722,
       0.03201141, 0.02751458, 0.02607392, 0.02385186, 0.02279782,
       0.01931497, 0.01888241, 0.01731148, 0.0168805 , 0.01593262,
       0.0142645 , 0.01391913, 0.01376722, 0.01294801, 0.01239229,
       0.01211645, 0.01170078, 0.01127194, 0.01113482, 0.01081091,
       0.01058115, 0.01040262, 0.01016023])

In [ ]:
display(train_t.shape)

(3156, 33)

In [ ]:
test_t = pca.transform(test_t)
test_t.shape

(1053, 33)

In [ ]:
import xgboost as xgb

xg_reg = xgb.XGBRegressor(objective ='reg:squarederror')
xg_reg.fit(train_t, y_train)

preds = xg_reg.predict(test_t)

In [ ]:
from sklearn.metrics import mean_squared_error

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 36.860231


In [ ]:
#CountFrequencyEncoder --- This has a encoding_method param. I will use frequency

encoder = CountFrequencyEncoder(encoding_method='frequency')

encoder.fit(X_train)

CFE_train_t= encoder.transform(X_train)
CFE_test_t= encoder.transform(X_test)

In [ ]:
display(CFE_train_t.shape)
CFE_test_t.shape
#no need for pca

(3156, 8)

(1053, 8)

In [ ]:
xg_reg.fit(CFE_train_t, y_train)
preds = xg_reg.predict(CFE_test_t)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 36.985782


In [ ]:
# OrdinalEcoder --- gives the category a number based on the mean of the target variable. Leaving the default of ordered values. 

encoder = OrdinalEncoder()

encoder.fit(X_train, y_train)

OE_train_t= encoder.transform(X_train)
OE_test_t= encoder.transform(X_test)

In [ ]:
display(OE_train_t.shape)
OE_test_t.shape

(3156, 8)

(1053, 8)

In [ ]:
xg_reg.fit(OE_train_t, y_train)
preds = xg_reg.predict(OE_test_t)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 36.918787


In [ ]:
# Mean Encoder -- This simply takes the mean of the target variable for that categorical variable

encoder = MeanEncoder()

encoder.fit(X_train, y_train)

ME_train_t= encoder.transform(X_train)
ME_test_t= encoder.transform(X_test)

display(ME_train_t.shape)
ME_test_t.shape

(3156, 8)

(1053, 8)

In [ ]:
xg_reg.fit(ME_train_t, y_train)
preds = xg_reg.predict(ME_test_t)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 36.918787


Note: the Weight of evidence encoder and PRratio encoder are only for binary classification. I will not be using it here. 

In [ ]:
encoder = DecisionTreeEncoder(encoding_method='ordered', random_state=42)

encoder.fit(X_train, y_train)

DCE_train_t = encoder.transform(X_train)
DCE_test_t = encoder.transform(X_test)

display(DCE_train_t.shape)
DCE_test_t.shape

(3156, 8)

(1053, 8)

In [ ]:
xg_reg.fit(DCE_train_t, y_train)
preds = xg_reg.predict(DCE_test_t)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 36.923739


### Encoder Conclusion

All the Encoders provided very similar RSME results. the lowest being 36.860231 and the highest 36.985782. So in this case the method of the encoder had very little effect. 

# Data Preparation 
 
The data has already been split by type. We are simply encoding, concatenating, splitting, scaling, and decomposing. In that order. 

In [ ]:
#despite knowing the performance is slightly better I wanted this one, its fewer features to deal with and I will already be reducing their number. 
encoder = CountFrequencyEncoder(encoding_method='frequency')
X_cat = encoder.fit_transform(X_cat)


In [ ]:
df1 = pd.concat([X_num, X_cat], axis=1)
df1.head()

,ID,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41,X42,X43,X44,X45,X46,X47,X48,X49,...,X353,X354,X355,X356,X357,X358,X359,X360,X361,X362,X363,X364,X365,X366,X367,X368,X369,X370,X371,X372,X373,X374,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385,X0,X1,X2,X3,X4,X5,X6,X8
0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.418389,0.096935,0.239249,0.104538,0.99905,0.63483,0.246852,0.596579
1,6,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.418389,0.222618,0.239249,0.052269,0.99905,0.63483,0.113566,0.596579
2,7,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,...,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.418389,0.222618,0.239249,0.461392,0.99905,0.63483,0.246852,0.596579
3,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.418389,0.222618,0.239249,0.255643,0.99905,0.63483,0.113566,0.053457
4,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.418389,0.096935,0.239249,0.255643,0.99905,0.63483,0.148491,0.057496


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df1, Y, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train, X_test)
SC_X_train = scaler.transform(X_train)
SC_X_test = scaler.transform(X_test)
SC_X_train.shape

(3156, 377)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.99)
pca.fit(SC_X_train)
SC_PCA_X_train = pca.transform(SC_X_train)
SC_PCA_X_test = pca.transform(SC_X_test)
pca.explained_variance_ratio_

array([0.07106197, 0.05695569, 0.04560657, 0.03471646, 0.03384955,
       0.03178491, 0.02867269, 0.02198505, 0.02003077, 0.01763912,
       0.01669874, 0.01608095, 0.01492914, 0.01445037, 0.01384108,
       0.01333553, 0.01307788, 0.0116078 , 0.01116144, 0.0109354 ,
       0.01006999, 0.00996702, 0.00989675, 0.00953577, 0.00910605,
       0.00867084, 0.00806059, 0.00788812, 0.00751735, 0.00730803,
       0.00707463, 0.00700068, 0.00668911, 0.00661622, 0.00640897,
       0.0060544 , 0.0059886 , 0.0058827 , 0.00569664, 0.00554556,
       0.00547205, 0.00537074, 0.00525021, 0.00517545, 0.00509262,
       0.00492101, 0.00477826, 0.00466006, 0.00454901, 0.00446441,
       0.00440907, 0.00434565, 0.00430046, 0.00426905, 0.00414929,
       0.00411887, 0.00399615, 0.00394003, 0.00392021, 0.00390028,
       0.00376293, 0.00370021, 0.0036813 , 0.00364669, 0.00358141,
       0.00351195, 0.00347813, 0.0034489 , 0.00344183, 0.00338236,
       0.00330678, 0.00330281, 0.00323624, 0.00321805, 0.00320

In [ ]:
SC_PCA_X_train.shape

(3156, 190)

### Intermediary thoughts

So we are making our way through our problem. In a technical sense we want that time prediction that regression will give us. And we are almost there. 

However

What we really want to know is how to reduce the time on the bench. That is the actual intent of this problem. When we look at the results of the PCA we can see that we have greatly reduced the number of features. But more importantly we can see that there are ~20 that explain most of the variance. That is 20 features explain >1% of the variance of the dataset. 

So therefore, if our goal is to walk into the plant and know which areas to look at, we already can create a list of the top 20 most likely culperates. 

In [ ]:
import xgboost as xgb

xg_reg = xgb.XGBRegressor(objective ='reg:squarederror')
xg_reg.fit(SC_PCA_X_train, y_train)

preds = xg_reg.predict(SC_PCA_X_test)

In [ ]:
from sklearn.metrics import mean_squared_error

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 9.112308


## Tuning
 
An RSME of 9.11 out of the box is much better than we saw with just the categorical terms. But lets play around with XGB and see if we can get an improvement. 
 
In this section I will run a number of different params and gridsearches to try to optimize this number. I used this section for as much for my own learning as trying to find the optimal number. 
 
After tuning we will use all this work and apply it to the "test" dataset with a pipeline. 


In [ ]:
xg_reg = xgb.XGBRegressor(booster='gblinear', objective ='reg:squarederror')
xg_reg.fit(SC_PCA_X_train, y_train)

preds = xg_reg.predict(SC_PCA_X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 8.762328


In [ ]:
xg_reg = xgb.XGBRegressor(booster='gblinear', objective ='reg:tweedie')
xg_reg.fit(SC_PCA_X_train, y_train)

preds = xg_reg.predict(SC_PCA_X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 8.536630


In [ ]:
dtrain = xgb.DMatrix(data=SC_PCA_X_train,label=y_train)
gbm_param_grid = {'learning_rate': [0.09,0.1,0.11], 'tweedie_variance_power':[1, 1.05, 1.09]}
gbm = xgb.XGBRegressor(booster='gblinear',objective="reg:tweedie", n_estimators= 200, subsample= 0.9)
grid_mse = GridSearchCV(estimator=gbm, param_grid=gbm_param_grid, scoring='neg_mean_squared_error', cv=4, verbose=1)
grid_mse.fit(SC_PCA_X_train, y_train)
print("Best parameters found: " ,grid_mse.best_params_)
print("Lowest RMSE found: " , np.sqrt(np.abs(grid_mse.best_score_)))

Fitting 4 folds for each of 9 candidates, totalling 36 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:   35.5s finished


Best parameters found:  {'learning_rate': 0.11, 'tweedie_variance_power': 1.09}
Lowest RMSE found:  12.384439387240471


In [ ]:
xg_reg = xgb.XGBRegressor(booster='gblinear', objective ='reg:tweedie', subsample=0.9, learning_rate=0.11, tweedie_variance_power=1.09)
xg_reg.fit(SC_PCA_X_train, y_train)

preds = xg_reg.predict(SC_PCA_X_test)
preds_train = xg_reg.predict(SC_PCA_X_train)

rmse_train = np.sqrt(mean_squared_error(y_train, preds_train))
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE Training: %f" % (rmse_train))
print("RMSE: %f" % (rmse))

RMSE Training: 8.201295
RMSE: 8.546445


In [ ]:
xg_reg = xgb.XGBRegressor(booster='gblinear', objective ='reg:tweedie')
xg_reg.fit(SC_PCA_X_train, y_train)

preds = xg_reg.predict(SC_PCA_X_test)
preds_train = xg_reg.predict(SC_PCA_X_train)

rmse_train = np.sqrt(mean_squared_error(y_train, preds_train))
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE Training: %f" % (rmse_train))
print("RMSE: %f" % (rmse))

RMSE Training: 8.218344
RMSE: 8.536630


In [ ]:
dtrain = xgb.DMatrix(data=SC_PCA_X_train,label=y_train)
gbm_param_grid = {'reg_lambda': [0, 0.5, 1, 1.5], 'tweedie_variance_power':[1, 1.1, 1.3, 1.5, 1.7, 1.9], 'reg_alpha':[0, 0.5, 1, 1.5]}
gbm = xgb.XGBRegressor(booster='gblinear',objective="reg:tweedie", n_estimators= 200)
grid_mse = GridSearchCV(estimator=gbm, param_grid=gbm_param_grid, scoring='neg_mean_squared_error', cv=4, verbose=1)
grid_mse.fit(SC_PCA_X_train, y_train)
print("Best parameters found: " ,grid_mse.best_params_)
print("Lowest RMSE found: " , np.sqrt(np.abs(grid_mse.best_score_)))

Fitting 4 folds for each of 96 candidates, totalling 384 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 384 out of 384 | elapsed:  6.0min finished


Best parameters found:  {'reg_alpha': 0, 'reg_lambda': 1, 'tweedie_variance_power': 1.9}
Lowest RMSE found:  8.81941331891472


In [ ]:
xg_reg = xgb.XGBRegressor(booster='gblinear', objective ='reg:tweedie', subsample=0.85, learning_rate=0.12, tweedie_variance_power=1.095)
xg_reg.fit(SC_PCA_X_train, y_train)

preds = xg_reg.predict(SC_PCA_X_test)
preds_train = xg_reg.predict(SC_PCA_X_train)

rmse_train = np.sqrt(mean_squared_error(y_train, preds_train))
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE Training: %f" % (rmse_train))
print("RMSE: %f" % (rmse))

RMSE Training: 8.201361
RMSE: 8.546235


In [ ]:
xg_reg = xgb.XGBRegressor(booster='gblinear', objective ='reg:tweedie', tweedie_variance_power=1.9, reg_lambda=1, n_estimators=200)
xg_reg.fit(SC_PCA_X_train, y_train)

preds = xg_reg.predict(SC_PCA_X_test)
preds_train = xg_reg.predict(SC_PCA_X_train)

rmse_train = np.sqrt(mean_squared_error(y_train, preds_train))
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE Training: %f" % (rmse_train))
print("RMSE: %f" % (rmse))

RMSE Training: 8.411827
RMSE: 8.657467


In [ ]:
dtrain = xgb.DMatrix(data=SC_PCA_X_train,label=y_train)
gbm_param_grid = {'tweedie_variance_power':[1, 1.1, 1.3, 1.5, 1.7, 1.9, 2], 'n_estimators':[100, 200, 300, 400] }
gbm = xgb.XGBRegressor(booster='gblinear',objective="reg:tweedie", n_estimators= 200, reg_lambda=1)
grid_mse = GridSearchCV(estimator=gbm, param_grid=gbm_param_grid, scoring='neg_mean_squared_error', cv=4, verbose=1)
grid_mse.fit(SC_PCA_X_train, y_train)
print("Best parameters found: " ,grid_mse.best_params_)
print("Lowest RMSE found: " , np.sqrt(np.abs(grid_mse.best_score_)))

Fitting 4 folds for each of 28 candidates, totalling 112 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 112 out of 112 | elapsed:  2.3min finished


Best parameters found:  {'n_estimators': 400, 'tweedie_variance_power': 1.7}
Lowest RMSE found:  8.7905888804261


In [ ]:
xg_reg = xgb.XGBRegressor(booster='gblinear', objective ='reg:tweedie', tweedie_variance_power=1.7, reg_lambda=1, n_estimators=400)
xg_reg.fit(SC_PCA_X_train, y_train)

preds = xg_reg.predict(SC_PCA_X_test)
preds_train = xg_reg.predict(SC_PCA_X_train)

rmse_train = np.sqrt(mean_squared_error(y_train, preds_train))
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE Training: %f" % (rmse_train))
print("RMSE: %f" % (rmse))

RMSE Training: 8.267229
RMSE: 8.554933


After a number of iterations, these are the parameters that I like the most. The results are fairly close together and one of the lowest I have seen yet. 
 
 
## Pipelines!
 
This project came with a separate dataset, competition style. So the next step is to take all that we just did and create a pipeline. Then predict the target variable, but with the letdown of not actually having the target variable for accuracy purposes. 
 
Note: I was disappointed with my pipeline. I really wanted to figure out how to fit some of these operations using the training dataset above, and transform on the test set below. I played around with caching transformers but couldn't get it. Clearly, I have my next learning objective for me. If you know the answer let me know!
 

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/Data Files/benz_test.csv'
df_test = pd.read_csv(path)
df_test.head()

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41,...,X345,X346,X347,X348,X349,X350,X351,X352,X353,X354,X355,X356,X357,X358,X359,X360,X361,X362,X363,X364,X365,X366,X367,X368,X369,X370,X371,X372,X373,X374,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,1,az,v,n,f,d,t,a,w,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,2,t,b,ai,a,d,b,g,y,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,3,az,v,as,f,d,a,j,j,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,4,az,l,n,f,d,z,l,n,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,5,w,s,as,c,d,y,i,m,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [ ]:
data = df_test.iloc[:, df_test.columns != 'ID']
id = df_test.iloc[:, df_test.columns == 'ID']
data.head()

,X0,X1,X2,X3,X4,X5,X6,X8,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41,X42,...,X345,X346,X347,X348,X349,X350,X351,X352,X353,X354,X355,X356,X357,X358,X359,X360,X361,X362,X363,X364,X365,X366,X367,X368,X369,X370,X371,X372,X373,X374,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,az,v,n,f,d,t,a,w,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,t,b,ai,a,d,b,g,y,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,az,v,as,f,d,a,j,j,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,az,l,n,f,d,z,l,n,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,w,s,as,c,d,y,i,m,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [ ]:
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import SimpleImputer

get_object = FunctionTransformer(lambda x: x.select_dtypes(include = 'object'))
get_int = FunctionTransformer(lambda x: x.select_dtypes(include = 'int64'))


pl = Pipeline([('union', FeatureUnion(transformer_list = [
                                                     ('categorical_features', Pipeline([
                                                              ('selector', get_object),
                                                              ('encoder', RareLabelEncoder(n_categories=3, replace_with='Rare')),
                                                              ('encoder_second', CountFrequencyEncoder(encoding_method='frequency')),
                                                     ])),
                                                     ('numeric_features', Pipeline([
                                                              ('selector', get_int),
                                                              ('imputer', SimpleImputer(missing_values=np.nan, strategy='mean'))
                                                     ]))    
                                                     ])),
                ('scaler', StandardScaler())
                
              ])



In [ ]:
pl.fit(df_test)
transformed_data = pl.transform(df_test)

In [ ]:
pca = PCA(n_components=0.99)
# SC_X_train is from the training set. 
pca.fit(SC_X_train)
#transforming the test set
scaled_data = pca.transform(transformed_data)

In [ ]:
xg_reg = xgb.XGBRegressor(booster='gblinear', objective ='reg:tweedie', tweedie_variance_power=1.7, reg_lambda=1, n_estimators=400)
#fitting the training set
xg_reg.fit(SC_PCA_X_train, y_train)
#predicting on the test set
preds = xg_reg.predict(scaled_data)
preds

array([ 96.909325,  94.90057 ,  95.59724 , ..., 101.45265 ,  98.170395,
        99.00827 ], dtype=float32)

In [ ]:
results = pd.concat([id,pd.DataFrame(preds, columns= ['Predictions'])], axis=1)
results

,ID,Predictions
0,1,96.909325
1,2,94.900574
2,3,95.597237
3,4,102.070984
4,5,98.830917
...,...,...
4204,8410,101.378517
4205,8411,97.029266
4206,8413,101.452652
4207,8414,98.170395


# Conclusion 
 
This dataset was pretty fun to play around with. I enjoyed the opportunity to explore some of the encoders and xgb params. However, I am left with more questions than I started with. 
 
How would I take this and help a plant manager improve their operations? It is one thing to be able to predict how long a vehicle will be sitting on the bench. It's another altogether to take that information and produce real world effects. I really wish I could take this and see it through to its final conclusion. 
